In [1]:
from instagram_private_api import Client, ClientCompatPatch
import api_requests
import safe_api
import importlib
importlib.reload(api_requests)
importlib.reload(safe_api)
from api_requests import (BasicUserInfo, PublicUserInfoRequests, MediaInfoRequests, 
                            InteractWithUsersActions, CollectDataThroughHashtags)
from safe_api import SafeClient
import time
import pdb

In [2]:
"""
Only run once, multiple log ins will cause account ban
TODO: Figure out how to cache auth cookie details. Hints: Can call api.cookie_jar, look into client.py
"""
username = 'cuteanimalzzzz123'
password = 'instabotOP'
api = SafeClient(username, password)

In [3]:
test_usernames = ['ryanlopezzzz', #small size public account example
                  'ruining99', #small size public account example
                  'i_am_jorge_murillo', #private account example
                  'cuteanimalzzzz123' #API login account
                 ]
large_test_usernames = ['cute.animals'] #large size public account

# Test BasicUserInfo() Class

In [4]:
def test_basic_user_info(username):
    """
    Can check against true values by going onto Instagram app.
    """
    basic_user_info = BasicUserInfo(api, username)
    print("User Id: " + str(basic_user_info.user_id))
    print("Follow count: " + str(basic_user_info.follower_count))
    print("Following count: " + str(basic_user_info.following_count))
    print("Media count: " + str(basic_user_info.media_count))
    print("Private status: " + str(basic_user_info.private_status))

In [5]:
for username in test_usernames + large_test_usernames:
    print("Username: " + username)
    test_basic_user_info(username)
    print("-"*100)

Username: ryanlopezzzz
User Id: 255272463
Follow count: 636
Following count: 391
Media count: 14
Private status: False
----------------------------------------------------------------------------------------------------
Username: ruining99
User Id: 2142436680
Follow count: 384
Following count: 371
Media count: 15
Private status: False
----------------------------------------------------------------------------------------------------
Username: i_am_jorge_murillo
User Id: 9984230653
Follow count: 176
Following count: 193
Media count: 2
Private status: True
----------------------------------------------------------------------------------------------------
Username: cuteanimalzzzz123
User Id: 50004795460
Follow count: 0
Following count: 1
Media count: 0
Private status: False
----------------------------------------------------------------------------------------------------
Username: cute.animals
User Id: 435102802
Follow count: 88151
Following count: 1
Media count: 344
Private status: F

# Test PublicUserInfoRequests() Class

In [10]:
def test_public_user_info_requests(username):
    puir = PublicUserInfoRequests(api,username)
    
    followers_usernames = puir.get_followers_usernames()
    assert(puir.follower_count == len(followers_usernames))
    print('Loaded %d follower usernames.'%len(followers_usernames))
    print('Peeking at first 3 followers: '+str(followers_usernames[:3]))
    print()    
    following_usernames = puir.get_following_usernames()
    assert(puir.following_count == len(following_usernames))
    print('Loaded %d following usernames.'%len(following_usernames))
    print('Peeking at first 3 following: '+str(following_usernames[:3]))
    print()
    user_feed_media_ids = puir.get_user_feed_media_ids()
    assert(puir.media_count == len(user_feed_media_ids))
    print('Loaded %d media (posts) ids.'%len(user_feed_media_ids))
    print('Peeking at first 2 ids: ' + str(user_feed_media_ids[:2]))

In [11]:
for username in test_usernames:
    print("Username: " + username + '\n')
    try:
        test_public_user_info_requests(username)
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print("-"*100)

Username: ryanlopezzzz

Loaded 636 follower usernames.
Peeking at first 3 followers: ['luca.scharrer', 'xubryana', 'c.hirish']

Exception: AssertionError, Message: 
----------------------------------------------------------------------------------------------------
Username: ruining99

Loaded 384 follower usernames.
Peeking at first 3 followers: ['luca.scharrer', 'xubryana', 'c.hirish']

Loaded 371 following usernames.
Peeking at first 3 following: ['luca.scharrer', 'xubryana', 'c.hirish']

Loaded 15 media (posts) ids.
Peeking at first 2 ids: ['2601326525749798387_2142436680', '2580166803319609035_2142436680']
----------------------------------------------------------------------------------------------------
Username: i_am_jorge_murillo

Exception: ValueError, Message: Can't get following info from private account.
----------------------------------------------------------------------------------------------------
Username: cuteanimalzzzz123

Exception: ValueError, Message: Strange be

# Test MediaInfoRequests() Class
Seems there can be an error where it loads less comments than get_comment_count() says there are.

In [12]:
def test_media_info_requests(username):
    #Get user's most recent media id
    puir = PublicUserInfoRequests(api,username)
    user_feed_media_ids = puir.get_user_feed_media_ids()
    most_recent_media_id = user_feed_media_ids[0]
    
    mir = MediaInfoRequests(api,most_recent_media_id)
    print('Comment count: %d'%mir.get_comment_count())
    print()                       
    comments_text = mir.get_n_comments_text()
    print('Loaded %d comments text.'%len(comments_text))
    print('Peeking at first 2 comments: '+str(comments_text[:2]))
    print()
    comment_counter = mir.get_n_comment_count_per_username()
    assert(sum(comment_counter.values())==len(comments_text)) 
    print('Loaded %d (username, comment count) data.'%len(comment_counter))
    comment_counter_list = list(comment_counter.items())
    print('Peeking at first 2 elements: '+str(comment_counter_list[:2]))

In [13]:
for username in test_usernames: #+ large_test_usernames:
    print("Testing on %s's first post: \n"%username)
    try:
        test_media_info_requests(username)
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print("-"*100)

Testing on ryanlopezzzz's first post: 

Comment count: 7

Exception: Warning, Message: _get_comments_info may not load all comments
----------------------------------------------------------------------------------------------------
Testing on ruining99's first post: 

Comment count: 2

Exception: Warning, Message: _get_comments_info may not load all comments
----------------------------------------------------------------------------------------------------
Testing on i_am_jorge_murillo's first post: 

Exception: ValueError, Message: Can't get following info from private account.
----------------------------------------------------------------------------------------------------
Testing on cuteanimalzzzz123's first post: 

Exception: ValueError, Message: Strange behavior when requesting your own info, needs further testing.
----------------------------------------------------------------------------------------------------


# Test InteractWithUsersActions() Class

In [14]:
def test_interact_with_users_actions(username):
    """
    Can check by looking onto the Instagram app when function is called
    """
    #Get user's id and oldest media id
    puir = PublicUserInfoRequests(api,username)
    user_id = puir.user_id
    user_feed_media_ids = puir.get_user_feed_media_ids()
    oldest_media_id = user_feed_media_ids[-1]
    
    iwua = InteractWithUsersActions(api)
    print('Following user %s...'%username)
    iwua.follow_user(user_id)
    print('Successfully followed - now sleeping')
    time.sleep(30) #can check that this user was followed in this time
    print()
    print('Unfollowing user')
    iwua.unfollow_user(user_id)
    print('Successfully unfollowed - now sleeping')
    time.sleep(10)
    print()
    print('Liking %s last media post'%username)
    iwua.like_post(oldest_media_id)
    print('Successfully liked - now sleeping')
    time.sleep(10)
    print()
    comment = 'Great photo!'
    print('Posting comment \'%s\' on %s last post'%(comment,username))
    iwua.post_comment(oldest_media_id,comment)
    print('Successfully posted comment.')

In [15]:
test_actions_username = test_usernames[0]
print('Testing on user %s: \n'%test_actions_username)
test_interact_with_users_actions(test_actions_username)

Testing on user ryanlopezzzz: 

Following user ryanlopezzzz...


ClientError: Bad Request: feedback_required

# Below is Uncompleted Code

In [ ]:
rank_token = api.generate_uuid()

In [ ]:
loaded_feed = api.feed_tag('banngood', rank_token)

In [ ]:
for key, value in loaded_feed.items():
    print(key)
    print(value)
    print('-'*80)

In [ ]:
print(len(loaded_feed['items']))
print(loaded_feed['items'][0])

In [ ]:
for key, value in loaded_feed['items'][0].items():
    print(key)
    print(value)
    print('-'*80)

In [ ]:
post_number = 2
print('Username:')
print(loaded_feed['items'][post_number]['user']['username'])